In [75]:
%load_ext heat

The heat extension is already loaded. To reload it, use:
  %reload_ext heat


In [82]:
from game_lists_site.models import (
    User,
    UserCBR,
    Game,
    UserGame,
    System,
    GameStats,
    GameDeveloper,
    GameGenre,
    GameTag,
    GameCBR,
)
from sklearn import preprocessing
import datetime as dt
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json
from operator import itemgetter
from game_lists_site.models import db
from game_lists_site.utils.utils import get_game_stats


def days_delta(datetime):
    if not datetime:
        return float("inf")
    return (dt.datetime.now() - datetime).days


def get_game_stats(game: Game):
    game_stats = GameStats.get_or_none(GameStats.game == game)
    if not game_stats or days_delta(game_stats.last_update_time) >= 7:
        game_stats, _ = GameStats.get_or_create(game=game)
        # features
        features = []
        features += [
            game_developer.developer.name.replace(" ", "")
            for game_developer in GameDeveloper.select(GameDeveloper.developer).where(
                GameDeveloper.game == game
            )
        ]
        features += [
            game_genre.genre.name.replace(" ", "")
            for game_genre in GameGenre.select(GameGenre.genre).where(
                GameGenre.game == game
            )
        ]
        features += [
            game_tag.tag.name.replace(" ", "")
            for game_tag in GameTag.select(GameTag.tag).where(GameTag.game == game)
        ]
        game_stats.features = " ".join(features)
        # features end
        users_game = UserGame.select(UserGame.playtime, UserGame.score).where(
            UserGame.game == game
        )
        users_game_with_playtime = users_game.where(UserGame.playtime > 0)
        users_game_with_score = users_game.where(UserGame.score > 0)
        game_stats.player_count = users_game_with_playtime.count()
        playtimes = np.array([ug.playtime for ug in users_game_with_playtime])
        scores = np.array([ug.score for ug in users_game_with_score])
        if len(playtimes) > 0:
            game_stats.total_playtime = np.sum(playtimes)
            game_stats.mean_playtime = np.mean(playtimes)
            game_stats.median_playtime = np.median(playtimes)
            game_stats.max_playtime = np.max(playtimes)
            game_stats.min_playtime = np.min(playtimes)
            if len(scores) > 2:
                game_stats.rating = np.mean(scores)
            else:
                game_stats.rating = 0
        game_stats.last_update_time = dt.datetime.now()
        game_stats.save()
    return game_stats


def get_cbr_for_game(game, result_count=9, min_player_count=10):
    system, _ = System.get_or_create(key="GameCBR")
    if not system.date_time_value or days_delta(system.date_time_value) >= 7:
        print("get_cbr_for_game")
        games = []
        features = []
        for gs in GameStats.select(GameStats.game, GameStats.features).where(
            GameStats.player_count >= min_player_count
        ):
            games.append(gs.game)
            features.append(gs.features)
        vectorizer = CountVectorizer()
        X = vectorizer.fit_transform(features)
        csr = cosine_similarity(X, X)  # cosine similarity result
        for g_a, row in zip(games, csr):
            l = 0
            precision = 0.7
            while l < 50:
                result = dict(
                    sorted(
                        [
                            (g_b.id, value)
                            for g_b, value in zip(games, row)
                            if value >= precision
                        ],
                        key=itemgetter(1),
                        reverse=True,
                    )
                )
                l = len(result)
                precision -= 0.05
            game_cbr, _ = GameCBR.get_or_create(game=g_a)
            game_cbr.data = json.dumps(result)
            game_cbr.save()
        system.date_time_value = dt.datetime.now()
        system.save()
    game_cbr = GameCBR.get_or_none(game=game)
    if game_cbr:
        data = {
            Game.get_by_id(game_id): value
            for game_id, value in json.loads(game_cbr.data).items()
        }
        return dict(list(data.items())[1 : result_count + 1])
    else:
        return {}

db.rollback()

user = User.get_by_id(76561198083927294)
user.last_cbr_update_time = None
user.save()
game = Game.get_by_id(412020)

In [89]:
def normalize_dict(dict_data: dict, coef: float = 1):
    values = list(dict_data.values())
    values = preprocessing.normalize([values])[0] * coef
    print(type(values))
    return {k: v for k, v in zip(dict_data, values)}


def get_cbr_for_user(user, played_user_games, result_count=9, cbr_for_game_result_count = 6):
    if not user.last_cbr_update_time or days_delta(user.last_cbr_update_time) >= 7:
        print("get_cbr_for_user")
        played_games = [ug.game for ug in played_user_games]
        user_games_with_score = played_user_games.where(UserGame.score != None)
        games_with_score = [ug.game for ug in user_games_with_score]
        result = {}
        for user_game, game_cbr_result in zip(
            user_games_with_score, [get_cbr_for_game(g, cbr_for_game_result_count) for g in games_with_score]
        ):
            if game_cbr_result:
                for sim_game in game_cbr_result:
                    if sim_game not in played_games and sim_game.rating >= 7:
                        if sim_game.id not in result:
                            result[sim_game.id] = (
                                user_game.score * game_cbr_result[sim_game]
                            )
                        else:
                            result[sim_game.id] += (
                                user_game.score * game_cbr_result[sim_game]
                            )
        user_cbr, _ = UserCBR.get_or_create(user=user)
        user_cbr.data = json.dumps(normalize_dict(
            dict(sorted(result.items(), key=lambda x: x[1], reverse=True)))
        )
        user_cbr.save()
        user.last_cbr_update_time = dt.datetime.now()
        user.save()
    data = {
        Game.get_by_id(game_id): value
        for game_id, value in json.loads(
            UserCBR.get_or_none(UserCBR.user == user).data
        ).items()
    }
    return dict(list(data.items())[:result_count])
played_user_games = (UserGame.select().where(UserGame.user == user).where(UserGame.playtime > 0))
print(get_cbr_for_user(user, played_user_games, result_count=9))

{<Game: 41700>: 0.23809738305999548, <Game: 412020>: 0.2285032633443253, <Game: 20510>: 0.1946559529988822, <Game: 205100>: 0.15359821859973363, <Game: 201870>: 0.1499439776037671, <Game: 729040>: 0.1423289438550046, <Game: 367500>: 0.13917657915056028, <Game: 460950>: 0.13264763097511184, <Game: 952060>: 0.1307438214439735}
